In [1]:
import openai
from openai import OpenAI
import pandas as pd
import json

In [2]:
key='API_KEY'
openai.api_key=key
client = OpenAI(api_key=key)

In [3]:
openai.models.list()

SyncPage[Model](data=[Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'), Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='gpt-4o-2024-05-13', created=1715368132, object='model', owned_by='system'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'), Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'), Model(id='text-embedding-3-small', created=

In [4]:
df=pd.read_csv('Food Data.txt',delimiter='\t', header=None)

In [5]:
print(df)

                                                    0
0   Hi, I'm Ravi. I love Pizza and Burger of LaPin...
1   Hey there, it's Sameer from Delhi. Butter Chic...
2   Namaste! I'm Kiran, a big fan of the Chole Bha...
3   What's up? This is Maya from Bengaluru. Can't ...
4   Hi everyone, it's Akash from Chennai. Huge fan...
5   Hey! I'm Neha from Mumbai. Love the Aloo Gobi ...
6   Hello! This is Rohan from Hyderabad. Biryani c...
7   What's up everyone? It's Aisha from Pune. Can'...
8   Hi there, I'm Dev from Chennai. South Indian f...
9   Namaste! This is Mira from Bengaluru. Nothing ...
10  Hey everyone, it's Rahul from Delhi. Weekends ...
11  What's up? I'm Nikita from Kolkata. Craving so...
12  Hi there, I'm Aditya from Mumbai. Love experim...
13  Hello! This is Priyanka from Delhi. Can't resi...
14  Hey everyone, it's Arjun from Chandigarh. The ...
15  What's up? I'm Shreya from Pune. Love a good f...
16  Hello! This is Kavita from Ahmedabad. Can't re...
17  Hey everyone, it's Sumit

In [6]:
data_array=df.values

In [7]:
def extract_user_info(message):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": message}
            ],
            functions=[
                {
                    "name": "user_info",
                    "description": "Get the user information from the text",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "name": {
                                "type": "string",
                                "description": "Name of the user"
                            },
                            "fav_food": {
                                "type": "string",
                                "description": "Favorite food of the user"
                            },
                            "fav_restaurant": {
                                "type": "string",
                                "description": "Restaurant name where users prefer to eat"
                            },
                            "total_amt": {
                                "type": "string",
                                "description": "Amount spent by the user monthly"
                            },
                            "city": {
                                "type": "string",
                                "description": "Location of user"
                            }
                        }
                    }
                }
            ]
        )
        return response
    except openai.error.OpenAIError as e:
        print(f"An error occurred: {e}")
        return None

In [13]:
def generate_suggestions(user_info):
    defaults = {
        'name': 'User',
        'fav_food': 'food',
        'fav_restaurant': 'restaurant',
        'city': 'city',
        'total_amt': 'some amount'
    }
    for key, value in defaults.items():
        user_info.setdefault(key, value)
    suggestion_prompt = (
        f"Suggest a restaurant name in {user_info['city']} that serves {user_info['fav_food']}. "
        "Please respond in the following JSON format:\n"
        "{\n"
        '  "restaurant_name": "Restaurant Name"\n'
        "}"
    )

    suggestion_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": suggestion_prompt}]
    )
    suggestions_text = suggestion_response.choices[0].message.content
    try:
        restaurant_info = json.loads(suggestions_text)
        restaurant_name = restaurant_info["restaurant_name"]
    except (json.JSONDecodeError, KeyError):
        restaurant_name = "None"
    print(restaurant_name)
    return restaurant_name

In [14]:
final_output = []

In [15]:
for message in data_array[:, 0]:
    user_info_response = extract_user_info(message)
    
    if user_info_response:
        user_info_str = user_info_response.choices[0].message.function_call.arguments
        print(user_info_str)
        user_info = eval(user_info_str)
        suggestions = generate_suggestions(user_info)
        restaurants = suggestions.strip().split('\n')
        first_suggestion = restaurants[0]
        final_output.append({
            'user': user_info,
            'suggestions': suggestions
        })

{"name":"Ravi","fav_food":"Pizza and Burger","fav_restaurant":"LaPinoz","total_amt":"Rs. 9000","city":"Jaipur"}
PizzaBurger Junction
{"name":"Sameer","fav_food":"Butter Chicken and Naan","fav_restaurant":"Zaika","total_amt":"Rs. 5000","city":"Delhi"}
Butter Naan Bistro
{"name":"Kiran","fav_food":"Chole Bhature","fav_restaurant":"Chandni Chowk","total_amt":"Rs. 2000","city":"Delhi"}
Chole Bhature Junction
{"name":"Maya","fav_food":"Rajma Chawal","fav_restaurant":"Biryani Paradise","total_amt":"Rs. 3000","city":"Bengaluru"}
Rajma Republic
{"name":"Akash","fav_food":"Thali meals","fav_restaurant":"Saravana Bhavan","total_amt":"Rs. 4000","city":"Chennai"}
Thali Tharavadu
{"name":"Neha","fav_food":"Aloo Gobi and Roti","fav_restaurant":"Ghar كي (Ki) Rasoi","total_amt":"Rs. 6000","city":"Mumbai"}
Aloo Gobi Junction
{"name":"Rohan","fav_food":"Biryani","fav_restaurant":"Paradise Biryani","total_amt":"Rs. 4500","city":"Hyderabad"}
Biryani Junction
{"name":"Aisha","fav_food":"Pav Bhaji","fav_res

In [17]:
print(final_output)

[{'user': {'name': 'Ravi', 'fav_food': 'Pizza and Burger', 'fav_restaurant': 'LaPinoz', 'total_amt': 'Rs. 9000', 'city': 'Jaipur'}, 'suggestions': 'PizzaBurger Junction'}, {'user': {'name': 'Sameer', 'fav_food': 'Butter Chicken and Naan', 'fav_restaurant': 'Zaika', 'total_amt': 'Rs. 5000', 'city': 'Delhi'}, 'suggestions': 'Butter Naan Bistro'}, {'user': {'name': 'Kiran', 'fav_food': 'Chole Bhature', 'fav_restaurant': 'Chandni Chowk', 'total_amt': 'Rs. 2000', 'city': 'Delhi'}, 'suggestions': 'Chole Bhature Junction'}, {'user': {'name': 'Maya', 'fav_food': 'Rajma Chawal', 'fav_restaurant': 'Biryani Paradise', 'total_amt': 'Rs. 3000', 'city': 'Bengaluru'}, 'suggestions': 'Rajma Republic'}, {'user': {'name': 'Akash', 'fav_food': 'Thali meals', 'fav_restaurant': 'Saravana Bhavan', 'total_amt': 'Rs. 4000', 'city': 'Chennai'}, 'suggestions': 'Thali Tharavadu'}, {'user': {'name': 'Neha', 'fav_food': 'Aloo Gobi and Roti', 'fav_restaurant': 'Ghar كي (Ki) Rasoi', 'total_amt': 'Rs. 6000', 'city': 

In [18]:
json_data=[]

In [19]:
for entry in final_output:
    user_info = entry['user']
    suggestions = entry['suggestions']
    json_data.append({
        'name': user_info['name'],
        'fav_food': user_info['fav_food'],
        'fav_restaurant': user_info['fav_restaurant'],
        'total_amt': user_info['total_amt'],
        'city': user_info['city'],
        'suggestions': suggestions
    })

In [20]:
df_user_info1 = pd.DataFrame(json_data)
print(df_user_info1)

        name                           fav_food            fav_restaurant  \
0       Ravi                   Pizza and Burger                   LaPinoz   
1     Sameer            Butter Chicken and Naan                     Zaika   
2      Kiran                      Chole Bhature             Chandni Chowk   
3       Maya                       Rajma Chawal          Biryani Paradise   
4      Akash                        Thali meals           Saravana Bhavan   
5       Neha                 Aloo Gobi and Roti        Ghar كي (Ki) Rasoi   
6      Rohan                            Biryani          Paradise Biryani   
7      Aisha                          Pav Bhaji                    Aadyaa   
8        Dev                  South Indian food                   Sankalp   
9       Mira                        Masala Dosa            Iyengar Bakery   
10     Rahul                  Mutton Rogan Josh                   Karim's   
11    Nikita                   Momos and Thukpa                 Doma Cafe   